In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# add project dir to python path
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    import sys, os
    sys.path.append("/Workspace" + os.path.dirname(os.path.dirname(notebook_path)))
except:
    pass

In [24]:
import pandas as pd
from coffee_weather import coffee_weather as cw

In [29]:
coffee_data = spark.read.table("dnb_wave_5.sandbox.df_arabica")

In [4]:
unique_regions = coffee_data.select("Country of Origin").distinct().collect()
region_list = [row["Country of Origin"] for row in unique_regions]

In [21]:
geocodes = cw.find_region_latlong(region_list)
latitude = [loc.latitude for location_dict in geocodes for country, loc in location_dict.items()]
longitude = [loc.longitude for location_dict in geocodes for country, loc in location_dict.items()]

location_weather_data = cw.pull_weather_data(latitude, longitude)

In [25]:
df = pd.DataFrame(location_weather_data)
df['date'] = df['date'].apply(lambda x: [date.date() for date in x])

spark_df = spark.createDataFrame(df)

In [26]:
extracted_info = [{"country": country, "latitude": loc.latitude, "longitude": loc.longitude} for location_dict in geocodes for country, loc in location_dict.items()]
region_location_map = spark.createDataFrame(extracted_info)

In [27]:
joined = spark_df \
    .join(region_location_map, spark_df.latitude == region_location_map.latitude, how='left') \
    .drop(region_location_map.latitude, region_location_map.longitude)

In [30]:
joined.write.mode("ignore").format("delta").saveAsTable("dnb_wave_5.sandbox.location_weather_raw")